In [1]:
## Import packages

from echopype.convert import Convert
from echopype import Process
import glob
import random
import matplotlib.pyplot as plt
import xarray as xr

import numpy as np

from os import path
import os


from echopy.processing import mask_seabed as  maskSB
from echopy.processing import mask_impulse as mIN
from echopy.processing import mask_shoals as mSH



from glider_azfp import load_glider, merge_glider_AZFP

In [2]:
# %%time # this times the cell

# # load in the glider data
# # get from erddap first time, then save locally

# gdata_local = './glider_data/'

# gdata_id ='ru32-20190102T1317-profile-sci-delayed'

# # check if the glider data is already saved
# if path.exists('./glider_data/'+ gdata_id + '.nc'):
    
#     gl = xr.open_dataset(gdata_local + gdata_id + '.nc')
    
# # if not, loaf from erddap and save it    
# else:

#     gl = load_glider(dataset_id = gdata_id)
    
#     if not path.isdir(gdata_local):
#         os.mkdir(gdata_local)
    
#     gl.load().to_netcdf(gdata_local + gdata_id + '.nc')
    

# # gl

In [2]:
%%time
gdata_local = './glider_data'
gdata_id='ru32-20190102T1317-profile-sci-delayed'
if path.exists('./glider_data/' + gdata_id + '.nc'):
    gl = xr.open_dataset(gdata_local + gdata_id + '.nc')
else:
    gl = load_glider(dataset_id = gdata_id)
    if not path.isdir(gdata_local):
        os.mkdir(gdata_local)
    gl.load().to_netcdf(gdata_local + gdata_id + '.nc')

Wall time: 197 ms


In [3]:
gl

<xarray.Dataset>
Dimensions:                  (profile: 719, time: 1421886)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-02T13:22:48.456450...
    latitude                 (time) float64 ...
    longitude                (time) float64 ...
    depth                    (time) float32 ...
Dimensions without coordinates: profile
Data variables:
    profile_id               (profile) float64 ...
    rowSize                  (profile) int32 ...
    trajectory               (time) object ...
    source_file              (time) object ...
    backscatter_coefficient  (time) float32 ...
    cdom                     (time) float32 ...
    chlorophyll_a            (time) float32 ...
    conductivity             (time) float32 ...
    crs                      (time) float64 ...
    ctd41cp_timestamp        (time) datetime64[ns] ...
    density                  (time) float32 ...
    flbbcd_timestamp         (time) datetime64[ns] ...
    instrument_azfp          (time) float64 ...
    instrument_ctd           (time) float64 ...
    instrument_flbbcd        (time) float64 ...
    m_pitch                  (time) float32 ...
    m_roll                   (time) float32 ...
    platform                 (time) float64 ...
    potential_temperature    (time) float32 ...
    pressure                 (time) float32 ...
    profile_lat              (time) float64 ...
    profile_lon              (time) float64 ...
    profile_time             (time) datetime64[ns] ...
    salinity                 (time) float32 ...
    sci_azfp_file_offset     (time) float32 ...
    sci_azfp_pause_signal    (time) float32 ...
    sci_azfp_run_state       (time) float32 ...
    sci_m_disk_free          (time) float32 ...
    sci_m_present_time       (time) datetime64[ns] ...
    sci_water_pressure       (time) float32 ...
    sound_speed              (time) float32 ...
    temperature              (time) float32 ...
    bottom_depth             (time) float32 ...
Attributes:
    cdm_data_type:                   Profile
    cdm_profile_variables:           profile_id
    comment:                         Deployed by Schuyler Nardelli and Nicole...
    contributor_name:                Oscar Schofield,Deborah Steinberg,Schuyl...
    contributor_role:                Principal Investigator,Principal Investi...
    Conventions:                     CF-1.6, COARDS, ACDD-1.3
    creator_email:                   kerfoot@marine.rutgers.edu
    creator_institution:             Rutgers University
    creator_name:                    John Kerfoot
    creator_type:                    person
    creator_url:                     https://rucool.marine.rutgers.edu
    date_created:                    2021-02-26T21:39:59Z
    date_issued:                     2021-02-26T21:39:59Z
    date_modified:                   2021-02-26T21:39:59Z
    defaultGraphQuery:               temperature,depth,time&time>=max(time)-1...
    deployment_id:                   555
    deployment_name:                 ru32-20190102T1317
    Easternmost_Easting:             -64.00849666666666
    featureType:                     Profile
    geospatial_bounds:               POLYGON ((-64.805435 -64.18056107402725,...
    geospatial_bounds_crs:           EPSG:4326
    geospatial_bounds_vertical_crs:  EPSG:5831
    geospatial_lat_max:              -64.805435
    geospatial_lat_min:              -64.87265000000001
    geospatial_lat_resolution:       0.00001 degree
    geospatial_lat_units:            degrees_north
    geospatial_lon_max:              -64.00849666666666
    geospatial_lon_min:              -64.18411333333333
    geospatial_lon_resolution:       0.00001 degree
    geospatial_lon_units:            degrees_east
    geospatial_verical_resolution:   0
    geospatial_vertical_max:         595.7142
    geospatial_vertical_min:         -0.4554963
    geospatial_vertical_positive:    down
    geospatial_vertical_units:       m
    history:                         2021-02-26T21:39:59Z:

In [3]:
## look at length of files ending with '.01A'
path_to_raw = ('./azfp_raw/')
all_files = glob.glob(path_to_raw+'*.01A') #need glob
calib_file = './azfp_raw/NLB_test.XML'

print(len(all_files), 'files') 
# all_files # this is the variable we want to loop over

385 files


In [5]:
# j = 0
# for banana in all_files:
    
#     print(j)
#     j = j + 1
#     fname = (banana)[7:-4] # gets rid of .01A
#     print(fname)

In [4]:
##### set some constant values


mean_T = gl.potential_temperature.mean().values # outside the loop
mean_S = gl.salinity.mean().values 
mean_P = gl.pressure.mean().values 


In [11]:
%%time

############ Processing Loop ###########################
########## The Mother of For Loops ######################
for banana in all_files[:]:   # for now just run first 3 files
    
    print(banana)
    
    ############ print all files ending in .01A #########
    
    fname = (banana)[11:-4]                                       # gets rid of .01A, have to alter indexing when changing directory
                                                                  # not fixed for all cases
    ###### convert raw azfp data into NetCDF ############
    
    data_tmp = Convert(path_to_raw + fname + '.01A', calib_file)  ## convert the raw azfp data
    savedir = './data_netcdfs/'                                   ## where processed files will live
    data_tmp.raw2nc(save_path = savedir )                         ## converts to a netcdf we can use with xarray
    nc_path = savedir + fname + '.nc'                             ## read the converted data
    ed = Process(nc_path)                                         ## create a processing object
    
    ######### assign temp/sal/pressure values ###########
    #### need to figure out how much it matter that these vary a lot
    
    ed.temperature = mean_T 
    ed.salinity = mean_S   
    ed.pressure = mean_P                                          ## pressure in dbar
    ed.recalculate_environment()                                  ## recalculate related parameters
    ed.calibrate()                                                ## Sv
    
    # Remove the first few bins with ringdown
    ed.Sv.Sv[:,:,:100] = np.NaN   # Turns data in first hundred bins to NaNs
    
    # SNR = signal to noise ratio; determines how much does Sv have to be to be significant
    # 2 is in dB (always starts from zero); Schuyler uses 10 (bigger the number the more data gets thrown out)
    
    ed.remove_noise(SNR=2)                                        ## denoise, this is adaptive! will change with the file! NOT WORKING NOW!
    azfp = ed.Sv_clean                                            ## assign name to cleaned dataset
    
    #### Subsetting, check if data is in subset ##########
    
    start_time = azfp.ping_time[0]
    end_time = azfp.ping_time[-1] 
    gds = gl.sel( time = slice(start_time,end_time) ) # select only the part that overlaps wt glider data
    
    ########## merge the glider and azfp data #############
    # count how many observation are there when the glide is pitched in the right range
    inds = (gds.m_pitch*180/np.pi < -15) & (gds.m_pitch*180/np.pi > -30) 
    if ( inds.sum() < 3):    
        print('')## check if glider data empty, used salinity but could use anything
        print(' no data, skipping AZFP file')
        print('')
        continue 
    else: # if first thing is not true, run the next line
        ds = merge_glider_AZFP( gl, azfp, )
    
    # some of the files are shorter than others
    # do an if statement: if ___ is true do ____
#     if (len(gds.salinity)<3):                                      ## check if glider data smaller than 3, used salinity but could use anything
#         print('no data, skipping AZFP file')
#         continue 
#     else: # if first thing is not true, run the next line
#         ds = merge_glider_AZFP( gl, azfp, )                        ## need to use the function merge_glider_AZFP that Nick made
                  
    # from the google
#     ds = xr.merge([gl, azfp]) # won't work bc it doesn't have the function

    ######### create mask for the bottom using echopy ######
    
    sf_mask = np.empty_like(ds.Sv)
    j = 0
    
    # create a nested loop
    for f in ds.frequency:           #loops over every frequency and is looking for the bottom
        sf_mask[j,:,:] = maskSB.ariza(ds.Sv.sel(frequency = f).data.T, ds.range[0,:,0].data, dc=3).T
        # T = transpose 
        j+=1
        #### look at default values in echopy documentation
    # create a bottom mask if any of the frequencies detect the bottom
    seafloor_mask = np.any(sf_mask, axis=0)                          ## looks @ sheets and values that are 1 to count it as the bottom 
    ds.update({'seafloor_mask': (('ping_time', 'range_bin'), seafloor_mask)})   
    
    # create a swarm mask
    # tried just adding one line in previous loop but ran into trouble, so this nested for loop could be temporary
    # need seafloor_mask in the dataset to make swarm mask
    # this takes a long time to run ...
    
    swarm_mask = np.empty_like(ds.Sv)
    j = 0 # this is not the jdim thing from echopy, it is how we stack the data
    
    for f in ds.frequency: # loops over ea. frequency
        swarm_mask[j,:,:] = mSH.echoview(ds.Sv.sel(frequency=f).where(~ds.seafloor_mask).data.T,
                                         np.arange( len(ds.Sv.range_bin )+1), # this is idim
                                         np.arange(len(ds.Sv.ping_time)+1), # this is jdim
                                         thr=-70, # play with threshold ; OG value = -85
                                         mincan=(5, 5), # parameters dont refer to units. mincan = minimum candidate
                                         maxlink=(30, 2),
                                         minsho=(30, 2) )[0].T
        # we set the minimum cadidate parameters for ____ and ____blank
        j+=1                                  
        ds.update({'swarm_mask': (('frequency','ping_time', 'range_bin'),swarm_mask)}) ;   
        
    ########### time to plot ###########################
    
    
    fig = plt.figure(figsize=(10.0, 4.0))
    svmin = -85              ## threshold min
    svmax = -30              ## threshold max
    
    f = ds.frequency[1]

    plt.subplot(131)
    ds.Sv.sel(frequency =f ).plot(x="distance", 
                                  y="bin_depths", yincrease=False, 
                                 vmin = svmin, vmax= svmax) # semicolon should suppress plot output
    plt.title('file number' + fname)

    plt.subplot(132)
    ds.Sv.sel(frequency=f).where(~ds.seafloor_mask).plot(x="distance", 
                                 y="bin_depths", yincrease=False, 
                                 vmin = svmin, vmax= svmax)
    plt.title('Sv no bottom')
    
    plt.subplot(133) # swarms mask
    # .where is when we use echopy info
    ds.Sv.sel(frequency=f).where(ds.swarm_mask.sel(frequency=f) # select backscatter(sV), one frequency; '.where' where something is true (where there's a swarm)
                                ).plot(x="distance", 
                                       y="bin_depths", yincrease=False, 
                                 vmin = svmin, vmax= svmax)
    plt.title('Sv in swarms')
    fig.tight_layout()
    plt.savefig('./figures/' + fname, dpi=300)            ## Path to file where figures will live
    plt.close()                                           ## this supresses the plot output
    
    ###### save merged dataset as netcdfs ############### (Only the 2 lines not commented are what's actually being used)

    savedir = './looped_netcdf/'                          ## place to save ds as a netcdf file
    ds.to_netcdf(savedir + fname + 'merged' + '.nc' )     ## Convert ds dataset into netcdf and save each filename    
    

./azfp_raw\19010212.01A
18:29:06 converting file 19010212.01A with NLB_test.XML, time of first ping 2019-Jan-02 12:53:15
          ... this file has already been converted to .nc, conversion not executed.
Initialize using average temperature recorded by instrument
18:29:06  calibrating data in ./data_netcdfs/19010212.nc
18:29:06  Remove noise from Sv stored in memory.

 no data, skipping AZFP file

./azfp_raw\19010213.01A
18:29:06 converting file 19010213.01A with NLB_test.XML, time of first ping 2019-Jan-02 13:24:45
          ... this file has already been converted to .nc, conversion not executed.
Initialize using average temperature recorded by instrument
18:29:07  calibrating data in ./data_netcdfs/19010213.nc
18:29:07  Remove noise from Sv stored in memory.
./azfp_raw\19010214.01A
18:29:13 converting file 19010214.01A with NLB_test.XML, time of first ping 2019-Jan-02 14:15:51
          ... this file has already been converted to .nc, conversion not executed.
Initialize using avera

ds.bottom_depth.plot(marker='o', ls='')

first pannel is raw data, second is applying the seafloor mask, third applies swarm mask

what other varibales need to be calculated (5x5m bins)

look @ Schuyler's processing notes 
    # in schulyter proccessing notes:Define krill swarms section, get correct parameters to put into swarm detection****
    
    i.	Minimum total school height: The Ross sea had a pulse length of 1000 µs. I used a minimum total swarm height of 0.75m after some trial and error.  
    c.	Minimum candidate length and height: i.	For the Ross Sea used minimum swarm candidate length of 0.5m and height of 0.3m after some trial and error. 
    
    d.	Maximum vertical and horizontal linking distance: i.	Because the Ross Sea deployment used a glider rather than a ship, used a maximum horizontal linking distance of 7m and a vertical linking distance of 5m after some trial and error. 
    
        # look at threshold 
        minimum threshold to -70 dB and maximum to -30 dB
        
        # min total school length
            a.	Minimum total school length: Calculate your minimum horizontal resolution by multiplying your ping interval (s) by your vessel’s standard cruising speed (m/s).
    i.	Gliders travel at about 0.38 m/s and the Ross Sea ping interval was 1 sec, so the minimum horizontal resolution ~ 0.38m. This means the minimum total swarm length ~ 0.75m. I used 1m for the Ross Sea data after some trial and error. 
    
        # look at echopy documentation for krill swarms and look at their values
        
        
        
        # Our units are in bins currently, hers are in meters, figure out bin size
        # how tall and wide is a bin
        # depth bin size and range bin size (distance cord; about .06m)
        
        ## draw out data to get an idea of what each bin looks like 

        #look @ ecopy code 
    
    # save ds at the end as a netcdf (in a folder just for those files b/c there's a lot)
        # look out for permission denied error
        # do the saving last
        
## in plotting, save the fig, give it a name wt file number in it
## add title wt name of AZFP file. plt.title(fname)
## there is a way to supress the code from producing the plots but will still save them (for running all the files)


In [8]:
### indexing :2 can mean up to but not including position 2
ds


<xarray.Dataset>
Dimensions:                   (frequency: 3, ping_time: 217, range_bin: 1999)
Coordinates:
  * frequency                 (frequency) int32 38000 125000 200000
  * ping_time                 (ping_time) datetime64[ns] 2019-01-02T15:00:01 ...
  * range_bin                 (range_bin) int32 0 1 2 3 ... 1995 1996 1997 1998
    bin_depths                (ping_time, range_bin) float64 34.05 ... 197.6
    distance                  (ping_time) float64 0.0 0.6696 ... 144.0 144.6
Data variables:
    Sv                        (frequency, ping_time, range_bin) float64 nan ....
    noise_est_range_bin_size  (frequency, ping_time) float64 5.016 ... 5.016
    range                     (frequency, range_bin, ping_time) float64 0.363...
    potential_temperature     (ping_time) float64 0.3203 0.3113 ... 0.803 0.8024
    salinity                  (ping_time) float64 33.83 33.83 ... 34.44 34.44
    chlorophyll_a             (ping_time) float64 0.442 0.4236 ... 0.1393 0.1531
    m_pitch                   (ping_time) float64 -0.398 -0.3993 ... -0.2736
    m_roll                    (ping_time) float64 -0.02835 -0.03328 ... -0.03823
    bottom_depth              (ping_time) float64 169.4 169.6 ... 215.0 215.1
    latitude                  (ping_time) float64 -64.81 -64.81 ... -64.81
    longitude                 (ping_time) float64 -64.18 -64.18 ... -64.18
    depth                     (ping_time) float64 33.68 34.11 ... 124.2 124.6
    seafloor_mask             (ping_time, range_bin) bool False False ... False
    swarm_mask                (frequency, ping_time, range_bin) float64 0.0 ....

In [ ]:
# determine what components of Schuyler's work I want to compare to mine
# one would be plot Sv from mine and Sv from hers and see how correlated they are
# Look at how to calculate bimass 9equation in her presentation 
# there is an intermediate step (area backscatter coeffiecent (sigma)) go from there to biomass
# determine what equations we need to do 
# in case study want to have figures of biomass 
# look at schuyler's notes and papers for the equations 
# utimately go from backscatter to biomass 
# writing and outlining 